In [1]:
#필요 라이브러리 임포트
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action='ignore')
plt.rcParams['axes.unicode_minus'] = False
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns

In [2]:
# 4319
train_dt = pd.read_csv('train.csv').set_index('id')
test_dt = pd.read_csv('test.csv').set_index('id')

In [3]:
print(train_dt.shape)
train_dt.head(3)

(4752, 59)


,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,...,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
id,,,,,,,,,,,,,,,,,,,,,
0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,...,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,NaN
1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,...,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,NaN
2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,...,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,NaN


In [4]:
new_columns = ['temperatures_00','local_air_pressure_01','wind_speed_02','wind_speed_03','daily_cumulative_precipitation_04',
               'barometric_pressure_05','local_air_pressure_06', 'temperatures_07','barometric_pressure_08','barometric_pressure_09',
               'daily_cumulative_precipitation_10','cumulative_daily_insolation_11','humidity_12','wind_direction_13','cumulative_daily_insolation_14',# 10-14
               'wind_direction_15', 'cumulative_daily_insolation_16','wind_direction_17', 'wind_speed_18','cumulative_daily_insolation_19',# 15-19
               'humidity_20', 'daily_cumulative_precipitation_21', 'local_air_pressure_22','barometric_pressure_23','wind_speed_24',# 20-24
               'wind_direction_25', 'wind_speed_26', 'local_air_pressure_27','temperatures_28','local_air_pressure_29', # 25-29
               'humidity_30', 'temperatures_31','temperatures_32','barometric_pressure_33','cumulative_daily_insolation_34', # 30-34
               'wind_direction_35', 'daily_cumulative_precipitation_36','humidity_37','humidity_38','daily_cumulative_precipitation_39',# 35-39
               'Y00', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'Y07', 'Y08', 'Y09','Y10', 'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y16', 'Y17', 'Y18'
              ]

train_dt.columns = new_columns
test_dt.columns = new_columns[:-19]

### 분석 방법

1. Y00 - Y17 값을 예측 / Y18 값과 유사한 데이터만 이용
    - 유사한 데이터 평균을 구해 Y18과 연결
    - 학습시키기

In [48]:
import lightgbm as lgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [50]:
X_train = train_dt.iloc[:, :-17]

X_train.head(2)

,temperatures_00,local_air_pressure_01,wind_speed_02,wind_speed_03,daily_cumulative_precipitation_04,barometric_pressure_05,local_air_pressure_06,temperatures_07,barometric_pressure_08,barometric_pressure_09,...,humidity_30,temperatures_31,temperatures_32,barometric_pressure_33,cumulative_daily_insolation_34,wind_direction_35,daily_cumulative_precipitation_36,humidity_37,humidity_38,daily_cumulative_precipitation_39
id,,,,,,,,,,,,,,,,,,,,,
0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,...,69.1,8.2,10.7,1010.1,0.0,256.4,0.0,77.2,62.6,0.0
1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,...,70.3,8.3,10.3,1010.1,0.0,215.4,0.0,77.3,63.5,0.0


In [75]:
X_train = train_dt.iloc[:4320, :40]
X_test = train_dt.iloc[4320:, :40]

In [76]:
def func(num):
    # -17 ~ -2 / -1 : Y18

    y_train = train_dt.iloc[:4320, 40 + num]
    y_test = train_dt.iloc[4320:, 40 + num]

    lgb_train = lgb.Dataset(X_train, label=y_train)

    lgb_param = {
      "objective":"regression",
      "metrics":"mae",
      "learning_rate":0.01
    }

    print("cv start")
    cv_result = lgb.cv(
      lgb_param,
      lgb_train,
      num_boost_round=99999,
      nfold=5,
      early_stopping_rounds=10,
      stratified=False,
      verbose_eval=10 
    )

    print("train start")
    lgb_model = lgb.train(
      lgb_param,
      lgb_train,
      num_boost_round=len(cv_result["l1-mean"])
    )

    pred = lgb_model.predict(X_test)

    return pred

In [77]:
main = np.array([])

for i in range(0,3):
    pred = func(i)
    if i:
        main = np.vstack([main, pred])
    else:
        main = np.copy(pred)
  
main = pd.DataFrame(main)

cv start
[10]	cv_agg's l1: 3.24016 + 0.0658918
[20]	cv_agg's l1: 2.93688 + 0.0608545
[30]	cv_agg's l1: 2.66315 + 0.0561457
[40]	cv_agg's l1: 2.41586 + 0.0512625
[50]	cv_agg's l1: 2.19265 + 0.0472214
[60]	cv_agg's l1: 1.99088 + 0.0436214
[70]	cv_agg's l1: 1.80885 + 0.0402595
[80]	cv_agg's l1: 1.64434 + 0.0373408
[90]	cv_agg's l1: 1.49548 + 0.0347194
[100]	cv_agg's l1: 1.36159 + 0.0321564
[110]	cv_agg's l1: 1.24099 + 0.0299439
[120]	cv_agg's l1: 1.13255 + 0.0281896
[130]	cv_agg's l1: 1.03503 + 0.0262403
[140]	cv_agg's l1: 0.947779 + 0.0244005
[150]	cv_agg's l1: 0.869472 + 0.0228469
[160]	cv_agg's l1: 0.799048 + 0.0215322
[170]	cv_agg's l1: 0.736105 + 0.0202269
[180]	cv_agg's l1: 0.679637 + 0.0189871
[190]	cv_agg's l1: 0.629135 + 0.0178121
[200]	cv_agg's l1: 0.584256 + 0.0168762
[210]	cv_agg's l1: 0.543957 + 0.0159082
[220]	cv_agg's l1: 0.508197 + 0.0151209
[230]	cv_agg's l1: 0.476622 + 0.0141205
[240]	cv_agg's l1: 0.448812 + 0.0132482
[250]	cv_agg's l1: 0.424347 + 0.0126929
[260]	cv_agg'

[2010]	cv_agg's l1: 0.206039 + 0.0031442
[2020]	cv_agg's l1: 0.206004 + 0.00314722
[2030]	cv_agg's l1: 0.205977 + 0.00314166
[2040]	cv_agg's l1: 0.20595 + 0.00313152
[2050]	cv_agg's l1: 0.20592 + 0.00311995
[2060]	cv_agg's l1: 0.205888 + 0.00312712
[2070]	cv_agg's l1: 0.205852 + 0.00312675
[2080]	cv_agg's l1: 0.205812 + 0.00313115
[2090]	cv_agg's l1: 0.205781 + 0.00313548
[2100]	cv_agg's l1: 0.205737 + 0.0031386
[2110]	cv_agg's l1: 0.205685 + 0.00315443
[2120]	cv_agg's l1: 0.205636 + 0.00315579
[2130]	cv_agg's l1: 0.20561 + 0.00316506
[2140]	cv_agg's l1: 0.205582 + 0.0031613
[2150]	cv_agg's l1: 0.205542 + 0.00315161
[2160]	cv_agg's l1: 0.205514 + 0.00313445
[2170]	cv_agg's l1: 0.20549 + 0.00313086
[2180]	cv_agg's l1: 0.205463 + 0.00312191
[2190]	cv_agg's l1: 0.20541 + 0.00309468
[2200]	cv_agg's l1: 0.205391 + 0.00308558
[2210]	cv_agg's l1: 0.20538 + 0.00308
[2220]	cv_agg's l1: 0.205339 + 0.00307973
[2230]	cv_agg's l1: 0.20533 + 0.00307532
[2240]	cv_agg's l1: 0.205305 + 0.00309147
[2250

[1320]	cv_agg's l1: 0.374826 + 0.0114726
[1330]	cv_agg's l1: 0.374503 + 0.0114162
[1340]	cv_agg's l1: 0.374181 + 0.0113234
[1350]	cv_agg's l1: 0.373828 + 0.0113091
[1360]	cv_agg's l1: 0.37351 + 0.0112178
[1370]	cv_agg's l1: 0.373184 + 0.0112223
[1380]	cv_agg's l1: 0.372855 + 0.0111775
[1390]	cv_agg's l1: 0.372547 + 0.0111346
[1400]	cv_agg's l1: 0.372254 + 0.01115
[1410]	cv_agg's l1: 0.371926 + 0.0110674
[1420]	cv_agg's l1: 0.371606 + 0.0110958
[1430]	cv_agg's l1: 0.371266 + 0.0110803
[1440]	cv_agg's l1: 0.370934 + 0.0110264
[1450]	cv_agg's l1: 0.370662 + 0.0109604
[1460]	cv_agg's l1: 0.370408 + 0.01095
[1470]	cv_agg's l1: 0.370155 + 0.0109354
[1480]	cv_agg's l1: 0.369872 + 0.0108929
[1490]	cv_agg's l1: 0.369626 + 0.0109089
[1500]	cv_agg's l1: 0.369384 + 0.010916
[1510]	cv_agg's l1: 0.369143 + 0.0108663
[1520]	cv_agg's l1: 0.368837 + 0.0108222
[1530]	cv_agg's l1: 0.368564 + 0.010783
[1540]	cv_agg's l1: 0.36832 + 0.0107111
[1550]	cv_agg's l1: 0.368074 + 0.0106499
[1560]	cv_agg's l1: 0.36

[3300]	cv_agg's l1: 0.350305 + 0.00839331
[3310]	cv_agg's l1: 0.350251 + 0.00841443
[3320]	cv_agg's l1: 0.35021 + 0.0084173
[3330]	cv_agg's l1: 0.350149 + 0.00842567
[3340]	cv_agg's l1: 0.350092 + 0.00839797
[3350]	cv_agg's l1: 0.350047 + 0.00838305
[3360]	cv_agg's l1: 0.350017 + 0.00838438
[3370]	cv_agg's l1: 0.349973 + 0.00838257
[3380]	cv_agg's l1: 0.349925 + 0.00837173
[3390]	cv_agg's l1: 0.349857 + 0.00837717
[3400]	cv_agg's l1: 0.349792 + 0.0083687
[3410]	cv_agg's l1: 0.349736 + 0.00836889
[3420]	cv_agg's l1: 0.349684 + 0.00836118
[3430]	cv_agg's l1: 0.349612 + 0.00834964
[3440]	cv_agg's l1: 0.349569 + 0.00833887
[3450]	cv_agg's l1: 0.349537 + 0.00832317
[3460]	cv_agg's l1: 0.34949 + 0.00831486
[3470]	cv_agg's l1: 0.349454 + 0.0082959
[3480]	cv_agg's l1: 0.349402 + 0.00828904
[3490]	cv_agg's l1: 0.349347 + 0.00827823
[3500]	cv_agg's l1: 0.349294 + 0.00825179
[3510]	cv_agg's l1: 0.349264 + 0.00824531
[3520]	cv_agg's l1: 0.349216 + 0.00824434
[3530]	cv_agg's l1: 0.349181 + 0.008251

[5260]	cv_agg's l1: 0.345105 + 0.00781974
[5270]	cv_agg's l1: 0.345091 + 0.00782261
[5280]	cv_agg's l1: 0.345081 + 0.00781823
[5290]	cv_agg's l1: 0.345069 + 0.00781577
[5300]	cv_agg's l1: 0.345058 + 0.00781299
[5310]	cv_agg's l1: 0.345045 + 0.00781395
[5320]	cv_agg's l1: 0.345029 + 0.00780688
[5330]	cv_agg's l1: 0.345016 + 0.00779846
[5340]	cv_agg's l1: 0.34501 + 0.00778659
[5350]	cv_agg's l1: 0.345001 + 0.00779215
[5360]	cv_agg's l1: 0.344994 + 0.00778126
[5370]	cv_agg's l1: 0.344983 + 0.00779296
train start
cv start
[10]	cv_agg's l1: 5.7755 + 0.0545824
[20]	cv_agg's l1: 5.23453 + 0.0494462
[30]	cv_agg's l1: 4.7467 + 0.0450159
[40]	cv_agg's l1: 4.30563 + 0.0404509
[50]	cv_agg's l1: 3.90759 + 0.0370937
[60]	cv_agg's l1: 3.54989 + 0.0347553
[70]	cv_agg's l1: 3.22684 + 0.0320634
[80]	cv_agg's l1: 2.93467 + 0.0297991
[90]	cv_agg's l1: 2.67192 + 0.0274476
[100]	cv_agg's l1: 2.43517 + 0.0259387
[110]	cv_agg's l1: 2.22124 + 0.0248554
[120]	cv_agg's l1: 2.02798 + 0.0233436
[130]	cv_agg's l1: 

[1930]	cv_agg's l1: 0.349717 + 0.0133958
[1940]	cv_agg's l1: 0.349546 + 0.0133546
[1950]	cv_agg's l1: 0.349433 + 0.0133491
[1960]	cv_agg's l1: 0.349308 + 0.0133069
[1970]	cv_agg's l1: 0.349149 + 0.0133052
[1980]	cv_agg's l1: 0.349032 + 0.0132807
[1990]	cv_agg's l1: 0.348891 + 0.0132413
[2000]	cv_agg's l1: 0.348741 + 0.0132022
[2010]	cv_agg's l1: 0.34862 + 0.0131925
[2020]	cv_agg's l1: 0.348493 + 0.0131767
[2030]	cv_agg's l1: 0.348322 + 0.0131685
[2040]	cv_agg's l1: 0.348165 + 0.0131245
[2050]	cv_agg's l1: 0.348053 + 0.0130676
[2060]	cv_agg's l1: 0.347949 + 0.0130274
[2070]	cv_agg's l1: 0.347827 + 0.0130149
[2080]	cv_agg's l1: 0.34769 + 0.0129949
[2090]	cv_agg's l1: 0.347592 + 0.0130059
[2100]	cv_agg's l1: 0.34748 + 0.0130089
[2110]	cv_agg's l1: 0.347343 + 0.0129947
[2120]	cv_agg's l1: 0.347201 + 0.0129869
[2130]	cv_agg's l1: 0.34707 + 0.0129595
[2140]	cv_agg's l1: 0.346966 + 0.0129678
[2150]	cv_agg's l1: 0.346846 + 0.0129678
[2160]	cv_agg's l1: 0.3467 + 0.0129502
[2170]	cv_agg's l1: 0.

[3930]	cv_agg's l1: 0.336462 + 0.0116525
[3940]	cv_agg's l1: 0.336427 + 0.0116442
[3950]	cv_agg's l1: 0.3364 + 0.0116355
[3960]	cv_agg's l1: 0.336372 + 0.0116226
[3970]	cv_agg's l1: 0.336333 + 0.011613
[3980]	cv_agg's l1: 0.336299 + 0.0116082
[3990]	cv_agg's l1: 0.336281 + 0.0116029
[4000]	cv_agg's l1: 0.336252 + 0.0115969
[4010]	cv_agg's l1: 0.336234 + 0.0116014
[4020]	cv_agg's l1: 0.336207 + 0.011594
[4030]	cv_agg's l1: 0.336184 + 0.0116079
[4040]	cv_agg's l1: 0.336151 + 0.0116096
[4050]	cv_agg's l1: 0.336128 + 0.0115987
[4060]	cv_agg's l1: 0.336104 + 0.0115971
[4070]	cv_agg's l1: 0.336094 + 0.0115989
[4080]	cv_agg's l1: 0.336084 + 0.0115958
[4090]	cv_agg's l1: 0.336066 + 0.0115964
[4100]	cv_agg's l1: 0.336057 + 0.011581
[4110]	cv_agg's l1: 0.336033 + 0.0115742
[4120]	cv_agg's l1: 0.336001 + 0.011566
[4130]	cv_agg's l1: 0.335978 + 0.011552
[4140]	cv_agg's l1: 0.335954 + 0.0115534
[4150]	cv_agg's l1: 0.335919 + 0.0115659
[4160]	cv_agg's l1: 0.335897 + 0.0115731
[4170]	cv_agg's l1: 0.3

In [83]:
main = main.T
main.head()

,0,1,2
0,21.264862,20.776083,20.901682
1,21.144733,20.691334,20.641416
2,20.984150,20.457191,20.341764
3,20.962650,20.279973,20.318783
4,20.945609,20.377343,20.271704


In [84]:
main.shape

(432, 3)

In [107]:
Y = train_dt.iloc[4320:,-1]

Y.index = main.index

In [109]:
result = pd.concat([main, Y], axis = 1)

result.corr()

,0,1,2,Y18
0,1.000000,0.961246,0.961037,0.712467
1,0.961246,1.000000,0.997409,0.829174
2,0.961037,0.997409,1.000000,0.832368
Y18,0.712467,0.829174,0.832368,1.000000
